In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#importing stuff: 
import numpy as np 
import matplotlib.pyplot as plt
import os 
import cv2

In [ ]:
#creating training and test sets
DATADIR = '/content/drive/MyDrive/Combined_1024'
CATEGORIES = ["Non-Pathologic", "Pathologic"]

In [ ]:
  #creating training dataset
  for category in CATEGORIES: 
    path = os.path.join(DATADIR, category) #gets us into our directory (dir) for each category 
    for img in os.listdir(path):
      img_array = cv2.imread(os.path.join(path, img)) #converts each image into an array, NOT CONVERTED TO GRAYSCALE BECAUSE COLOR MATTERS
  #creating training dataset

  training_data = []
  IMG_SIZE = 224
  new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))

  def create_training_data():
    for category in CATEGORIES: 
      path = os.path.join(DATADIR, category) #gets us into our directory (dir) for each category 
      class_num = CATEGORIES.index(category) #assigning a number as a label for each dataset
      for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path, img)) 
        new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
        training_data.append([new_array, class_num])

  create_training_data()

  print(len(training_data))



KeyboardInterrupt: ignored

In [ ]:
 #shuffling the data
import random 
random.shuffle(training_data)
X = []
y = []
for features, label in training_data: 
  X.append(features)
  y.append(label)
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

NameError: ignored

In [ ]:
import pickle
pickle_out = open("X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

X = pickle.load(open("X.pickle", "rb"))
y = pickle.load(open("y.pickle", "rb"))

X = X/255.0

model = Sequential()

model.add(Conv2D(256, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X, y, batch_size=32, epochs=3, validation_split=0.3) 

NameError: ignored

In [ ]:
#coding VGG16
import keras
import os
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense

VGG = keras.applications.VGG16(input_shape=(224,224,3),include_top=False,weights='imagenet')
VGG.trainable = False

model = keras.Sequential([
  VGG, 
  keras.layers.Flatten(),
  keras.layers.Dense(units = 256,activation="relu"),
  keras.layers.Dense(units = 256,activation="relu"),
  keras.layers.Dense(units=2, activation="softmax")
])

model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])
model.summary()
hist = model.fit(X, y, batch_size=16, epochs = 10, validation_split=0.1)

In [ ]:
#visualizations
plt.plt(hist.history["accuracy"])
plt.plot(hist.history['val_accuracy'])
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title("model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy", "Validation Accuracy", "loss", "Validation Loss"])

In [ ]:
#installing stuff

In [ ]:
!nvidia-smi

In [ ]:
#generating a confusion matrix 
from sklearn.metrics import classification_report, confusion_matrix
